In [49]:
# 전처리 라이브러리
import pandas as pd
import numpy as np
import os

# 작업 진행바 불러오기
from tqdm import tqdm

In [51]:
os.chdir('/content/drive/Shareddrives/강서구/220315 DrawMap/data/')
resident = pd.read_csv('행정구역_읍면동_별_5세별_주민등록인구_2011년__20220317090236.csv', encoding='cp949')

# 강서구 행정동별 주민등록인구 (2월)
res = pd.concat([resident[['행정구역(동읍면)별','항목']], resident.iloc[:,46:]], axis=1)
res.columns = res.loc[0,].tolist()
res.drop([0,1,2,3,4,5,6], axis=0, inplace=True)
res["행정구역(동읍면)별"] = res["행정구역(동읍면)별"].str.replace(pat=r'제', repl=r'', regex=True)

res_all = res[res['항목']=='총인구수 (명)'].reset_index(drop=True).drop('항목', axis=1)
res_all.set_index('행정구역(동읍면)별',inplace=True)
res_all = res_all.astype('float64')
for i in range(0,10):
    res_all[str(i*10)+'-'+str(i*10+10)+'세'+'_주민'] = res_all.iloc[:,(1+2*i):(3+2*i)].sum(axis=1)

res_all = pd.concat([res_all.iloc[:,0],res_all.iloc[:,22:],res_all.iloc[:,21]], axis=1)
res_all.reset_index(inplace=True)

res_all = pd.concat([res_all.iloc[:,0],res_all.iloc[:,3:9]], axis=1)
res_all['10-60세_주민'] = res_all.iloc[:,1:6].sum(axis=1)
res_all.rename(columns={'행정구역(동읍면)별':'동'}, inplace=True)
res_all = res_all[['동','10-60세_주민']]

In [52]:
res_all.to_csv('/content/drive/Shareddrives/강서구/using_data/주민등록.csv', index=False)

In [53]:
# 생활인구 데이터 불러오기
os.chdir('/content/drive/Shareddrives/강서구/220311 EDA/행정구 단위/')
df_people = pd.read_csv('LOCAL_PEOPLE_DONG_202202.csv')

# 한글 columns명이 너무 길어서 영어명(생활인구_스키마_공개 파일 기준)으로 간소화
df_kor = df_people.columns
df_people.reset_index(inplace=True)
df_people = df_people.drop(['여자70세이상생활인구수'], axis=1) # 한칸씩 밀리는 오류 해결
df_people.columns = ['YMD','TT','H_DNG_CD','SPOP',
                       'M00','M10','M15','M20','M25','M30','M35','M40','M45','M50','M55','M60','M65','M70',
                       'F00','F10','F15','F20','F25','F30','F35','F40','F45','F50','F55','F60','F65','F70']

# 강서구의 행정코드는 11500510~11500641
gs_people = df_people[(df_people['H_DNG_CD'] >= 11500510) & (df_people['H_DNG_CD'] <= 11500641)]
gs_people = gs_people.reset_index(drop=True)

# 행정동 이름 및 코드 딕셔너리
gs_lg = ['염창동','등촌1동', '등촌2동', '등촌3동', '화곡1동', '화곡2동', '화곡3동', '화곡4동', '화곡본동', '화곡6동', '화곡8동',
         '가양1동', '가양2동', '가양3동', '발산1동', '우장산동', '공항동','방화1동','방화2동','방화3동']
gs_cd = [11500510,11500520,11500530,11500535,11500540,11500550,11500560,11500570,11500590,11500591,
         11500593,11500603,11500604,11500605,11500611,11500615,11500620,11500630,11500640,11500641]
gs_dict =  {gs_cd[i] : gs_lg[i] for i in range(len(gs_lg))}

gs_m = gs_people.iloc[:,:18]
gs_f = pd.concat([gs_people.iloc[:,0:4],gs_people.iloc[:,18:]], axis=1)

def ten_sum(df):
    for i in range(1,7):
        df[str(i*10)+'-'+str(i*10+10)+'세'+'_생활'] = df.iloc[:,(3+2*i):(5+2*i)].sum(axis=1)

ten_sum(gs_m)
gs_m = pd.concat([gs_m.iloc[:,0:3],gs_m.iloc[:,18:]], axis=1)
ten_sum(gs_f)
gs_f = pd.concat([gs_f.iloc[:,0:3],gs_f.iloc[:,18:]], axis=1)

gs_m['H_DNG_CD'] = gs_m['H_DNG_CD'].apply(lambda x : gs_dict.get(x) if x else x)
gs_f['H_DNG_CD'] = gs_f['H_DNG_CD'].apply(lambda x : gs_dict.get(x) if x else x)

gs_m = gs_m.groupby(['H_DNG_CD']).mean() # 행정동별 평균값
gs_m.drop(['TT','YMD'], axis=1, inplace=True)
gs_m.reset_index(inplace=True)

gs_f = gs_f.groupby(['H_DNG_CD']).mean() # 행정동별 평균값
gs_f.drop(['TT','YMD'], axis=1, inplace=True)
gs_f.reset_index(inplace=True)

gs = pd.concat([gs_f, gs_m]).groupby(['H_DNG_CD']).sum().reset_index()
gs.rename(columns={'H_DNG_CD':'동'}, inplace=True)
gs['10-60세_생활'] = gs.iloc[:,1:6].sum(axis=1)
gs = gs[['동','10-60세_생활']]

In [54]:
gs.to_csv('/content/drive/Shareddrives/강서구/using_data/생활인구.csv', index=False)

In [55]:
# 지하철 데이터 불러오기
subway = pd.read_csv('/content/drive/Shareddrives/강서구/220319 MakeDF/data/서울시 지하철호선별 역별 승하차 인원 정보.csv')
sub_col = subway.columns
subway.reset_index(inplace=True)
subway = subway.iloc[:,:-1]
subway.columns = sub_col

subway = subway[(subway['역명']=="방화")|(subway['역명']=="개화산")|(subway['역명']=="송정")|(subway['역명']=="마곡")|(subway['역명']=="발산")|
                (subway['역명']=="우장산")|(subway['역명']=="화곡")|(subway['역명']=="까치산")|(subway['역명']=="김포공항")|(subway['역명']=="공항시장")|
                (subway['역명']=="신방화")|(subway['역명']=="마곡나루")|(subway['역명']=="양천향교")|(subway['역명']=="가양")|(subway['역명']=="증미")|
                (subway['역명']=="개화")]

subway = subway.groupby('역명').sum()
subway.reset_index(inplace=True)

subway_dic = {"방화":[37.577446,126.812741], "개화산":[37.572399,126.806171], "송정":[37.561184,126.811973], "마곡":[37.560183,126.825448],
              "발산":[37.558598,126.837668], "우장산":[37.548768,126.836318], "화곡":[37.541513,126.840461], "까치산":[37.531768,126.846683],
              "김포공항":[37.562434,126.801058], "공항시장":[37.563726,126.810678], "신방화":[37.567532,126.816601], "마곡나루":[37.567336,126.829497],
              "양천향교":[37.568381,126.841333], "가양":[37.561391,126.854456], "증미":[37.557402,126.861939],
              "개화":[37.578608,126.798153]}
subway['좌표'] = subway['역명'].apply(lambda x : subway_dic.get(x) if x else x)
subway['위도'] = subway['좌표'].str[0]
subway['경도'] = subway['좌표'].str[1]


subway = subway[['역명','승차총승객수', '하차총승객수', '위도', '경도']]

In [56]:
import requests
from six.moves import urllib
from urllib.request import urlopen, Request
from urllib.parse import quote
import json

In [57]:
# 네이버 reverse geocoding API
def rev_gc(code): # 경도,위도 형태
  client_id = "eylhc2t9ca"
  client_secret = "We65mz445phBKswlUP2BAC6ykiacLfDikseAdLvQ"
  code = urllib.parse.quote(code)
  url = 'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords=%s&sourcecrs=epsg:4326&orders=admcode&output=json'%(code)
  request = urllib.request.Request(url)
  request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
  request.add_header("X-NCP-APIGW-API-KEY",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()
  if(rescode==200):
      result = json.loads(response.read().decode('utf-8'))
      dong = result['results'][0]['region']['area3']['name']
  else:
      print("Error Code:" + rescode)
  return dong

In [58]:
subway['행정동명'] = 0
for i in range(len(subway)):
    subway['행정동명'][i] = rev_gc(str(subway['경도'][i])+","+str(subway['위도'][i]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [59]:
subway['승하차수'] = subway[['승차총승객수','하차총승객수']].sum(axis=1)
subway = subway[['행정동명','역명','승하차수']].groupby('행정동명').sum().reset_index()
subway.rename(columns={'행정동명':'동'}, inplace=True)

In [60]:
subway.to_csv('/content/drive/Shareddrives/강서구/using_data/강서구_지하철_승하차.csv', index=False)

In [ ]:
subway

,행정동명,승하차수
0,가양1동,405527
1,공항동,867366
2,등촌1동,303483
3,등촌3동,861457
4,발산1동,1476582
5,방화1동,291037
6,방화2동,1674322
7,방화3동,280825
8,화곡1동,2453601
